<a href="https://colab.research.google.com/github/tschnormeier/tf-cv1/blob/main/C1_W3_Lab_1_improving_accuracy_using_convolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
print(tf.__version__)


ModuleNotFoundError: No module named 'tensorflow'

# Ungraded Lab: Improving Computer Vision Accuracy using Convolutions
# Shallow Neural Network
(Course 1, Week 3, Reading4 -Try it yourself (Lab 1) use a convolutional network with FMNIST dataset)
In the previous lessions, you saw how to do fashion recognition using a neural network containing three layers, - the input layer (in the shape of the data), the output layer (in the shape of the desired output) and only one hidden layer.  You experimented with the impact of dirrerent sizes of hidden layer, number of training epochs etc on the final accuracy.  For convenience, here's the entire code again.  Run it and take a note of the test accuracy that is printed out at the end.
(original file found at URL: https://github.com/https-deeplearning-ai/tensorflow-1-public/blob/main/C1/W3/ungraded_labs/C1-W3_Lab_1_improving_accuracy_using_convolutions.ipynb )

In [ ]:
import tensorflow as tf


import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
print(tf.__version__)


In [ ]:
# Define a call back class
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') < 0.4):
            print("\nLoss is low is cancelling training!")
            self.model.stop_training = True

In [ ]:
# Instantiate the call back class
callbacks = myCallback()

## Load the fashion MNIST dataset

In [3]:
# Load fashion MNIST dataset and split into training and test sets
fmnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = fmnist.load_data()
# (start of extra code from prior lab)
# you can put between 0 to 59999 here
index = 2
# set the number of characters per row to display when printing
np.set_printoptions(linewidth=320)  # for 28 bytes per line of output

NameError: name 'tf' is not defined

In [ ]:
# define the Fashion dataset names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Visualize an image

In [ ]:
# print the label and image
print(f'LABEL: {training_labels[index]} is a {class_names[training_labels[index]]}')
print(f'\nImage Pixel Array:\n {training_images[index]}')
# visualize an image
plt.imshow(training_images[index], cmap='Greys')

In [ ]:
# display first 25 images
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(training_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[training_labels[i]])
plt.show()
# (end of extra code from prior lab)

In [ ]:
# Normalize the pixel values of the training and testing images
training_images = training_images / 255.0
test_images  = test_images  / 255.0


In [ ]:
# Build a simple Sequential model
model = tf.keras.models.Sequential([ tf.keras.layers.Flatten(),
                                     tf.keras.layers.Dense(units=128, activation = tf.nn.relu),
                                     tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [ ]:
# Compile the model - Setup training parameters
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# modify the call to the fit() to use the callback
# Train the model
print(f'\nMODEL TRAINING:')
# model.fit(training_images, training_labels, epochs=5, callbacks = [callbacks])
model.fit(training_images, training_labels, epochs=5 )

In [ ]:
# Evaluate on the test set
print(f'\nMODEL EVALUATION')
test_loss = model.evaluate(test_images, test_labels)

# Convolutional Neural Network
In the model above, your accuracy will probably be about 89% on training and 87% on validation.  Not bad.  But how do you make that even better?  One way is to use something called convolutions.  We're not going into the details of convolutions in this notebook (please see resources in the classroom), but the ultimate concept is that they narrow down the content of the image to focus on specific parts and this will likely improve the model accuracy.

if you've ever done image processing using a filter 
(like this URL: "https://en.wikipedia.org/wiki/Kernel_(image_processing" ) then convolutions will look very familiar.  In short, you take an array (usually 3x3 or 5x5) and scan it over the entire image.  By changing the underlying pixels based on the formula within that matrix, you can do things like edge detection.  So, for example, if you look at the above link, you'll see a 3x3 matrix that is defined for edge detection where the middle cell is 8, and all of its neighbors are -1.  In this case, for each pixel, you would multiply its value by 8, then subtract the value of each neighbor.  Do this for every pixel, and you'll end up with a new image that has the edges enhanced.

This is perfect for computer vision because it often highlights features that distinguish one item from another.  Moreover, the amount of information needed is then much less because you'll just train on the highlighted features.

That's the concept of Convolutional Neural Networks.  Add some layers to do convolutions before you have the dense layers, and then the information going to the dense layers is more focused and possibly more accurate.

Run the code below.  This is the same neural network as earlier, but this time with Convolution and MaxPooling layers added first.  It will take longer, but look at the impact on the accuracy.

In [ ]:
# Define the model using a CNN
model = tf.keras.models.Sequential([
                    # Add convolutions and max pooling layers
                    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
                    tf.keras.layers.MaxPooling2D(2, 2),
                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                    tf.keras.layers.MaxPooling2D(2,2),
                    # Add the same layers as before
                    keras.layers.Flatten(),
                              keras.layers.Dense(units=512, activation = tf.nn.relu),
                              keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Use the same settings
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
print(f'\nMODEL TRAINING:')
model.fit(training_images, training_labels, epochs=5)

# Evaluate on the test set
print(f'\nMODEL EVALUATION:')
test_loss = model.evaluate(test_images, test_labels)

It's likely gone up to about 92% on the training data and 90% on the validation data.  That's significant, and a step in the right direction!
Look at the code again, and see, step by step how the convolutions were built.  Instead of the input layer at the top, you added a Conv2D layer.
The parameters are:
1) The number of convolutions you want to generate.  The value here is purely arbitrary but it's good to use powers of 2 starting from 32.
2) The size of the Convolution.  In this case, a 3x3 grid.
3) The activation function to use.  In this case, you used a ReLU, which you might recall is the equivalent of returning x when x > 0, else return 0.
4) In the first layer, the shape of the input data.

You'll follow the convolution with a MaxPool2D layer which is designed to compress the image, while maintaining the content of the features that were highlighted by the convolution.  By specifying (2,2) for the MaxPooling, the effect is to quarter the size of the image.  Without going into too much detail here, the idea is that it creates a 2x2 array of pixels, and picks the biggest one (i.e. the Max pixel).  Thus, it turns 4 pixels into 1.  It repeats this across the image, and in doing so, it halves both the number of horizontal and vertical pixels, efectively reducing the image to 25% of the original image.

You can call model.summary() to see the size and shape of the network, and you'll notice that after every max polling layer, the image size is reduced in this way.

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1),
  tf.keras.layers.MaxPooling2D(2,2),

Then you added another convolution, pooling, and flattened the output.

  tf.keras.layers.Conv2E(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),

After this, you'll just have the same DNN structure as the non convolutional version.  The same 128 dense layers, and 10 output layers as in the pre-convolution example.

  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense( 10, activation='softmax')
])

# About Overfitting
Try running the training for more epochs - say about 20, and explore the results.  But while the results might seem really good, the validation results may actually go down, due to something called overfitting.  In a nutshell, overfitting occurs when the network learns the data from the training set really well, but it's too specialised to only that data, and as a reult is less effective at interpreting other unseen data.  For example, if all your life you only saw red shoes, then when you see a red shoe you would be very good at identifying it.  But a blue suede shoes might confuse you... and yo know you should never mess with my blue suede shoes.
# Visualizing the Convolutions and Pooling
Let's explore how to show the convolutions graphically.  The cell below prints the first 100 labels in the test set, and you can see that the ones at index 0, index 23, and index 28 are all the same value (i.e. 9).  They're all shoes.  Let's take a look at the result of running the convolution on each, and you'll begin to see common features betwen them emerge.  Now, when the dense layer is training on that data, it's working with a lot less, and it's perhaps finding a commonality between shoes based on this convolution / pooling combination.



In [ ]:
# add two lines of hearders for help count the index values
h1, h2 = "*", "*"
for i in range(0, 10):
    h1 = h1 + ( 10 * (str(i) + " "))
for i in range(0, 100):
    h2 = h2 + ( str(i%10) + " ")
print(h1);print(h2)

print(test_labels[:100])


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras import models

f, axarr = plt.subplots(3,4)

FIRST_IMAGE = 0
SECOND_IMAGE = 23
THIRD_IMAGE = 28
CONVOLUTION_NUMBER = 1

layer_outputs = [layer.output for layer in model.layers]
activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)

for x in range(0,4):
    f1 = activation_model.predict(test_images[FIRST_IMAGE].reshape(1, 28, 28, 1))[x]
    axarr[0,x].imshow(f1[0, :, :, CONVOLUTION_NUMBER], cmap='inferno')
    axarr[0,x].grid(False)
    
    f2 = activation_model.predict(test_images[SECOND_IMAGE].reshape(1, 28, 28, 1))[x]
    axarr[1,x].imshow(f2[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
    axarr[1,x].grid(False)
    
    f3 = activation_model.predict(test_images[THIRD_IMAGE].reshape(1, 28, 28, 1))[x]
    axarr[2,x].imshow(f3[0, : , : , CONVOLUTION_NUMBER], cmap='inferno')
    axarr[2,x].grid(False)

# Exercises
1. Try editing the convolutions.  Change the 32x to either 16 or 64.  What impact will this have on accuracy and / or training time.

2. Remove the final Convolution.  What impact will this have on accuracy or training time?

3. How about adding more Convolutions?  What impact do you this this will have?  Experiment with it.

4. Remove all Convolutions but the first.  What impact do you think this will have?  Experiment with it.

5. In the previous lesson you implemented a callback on the loss function and to cancel training once it hit a certain amount.  See if you can implement that here.
